In [4]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import pickle
import matplotlib.pyplot as plt
import timm

In [5]:
def accuracy(output, target):
    with torch.no_grad():
        pred = torch.argmax(output, dim=1)
        assert pred.shape[0] == len(target)
        correct = 0
        correct += torch.sum(pred == target).item()
    return correct / len(target)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with open('images_path.pickle','rb') as f:
    images_path = pickle.load(f)
    
batch_size = 64
MODEL_PATH ="saved"
folder_path = '/opt/ml/input/data/train/images/'
LEARNING_RATE = 0.001
EPOCHS = 10

In [7]:
class StateClassifier(nn.Module):
    def __init__(self, num_of_classes = 3):
        super().__init__()
        self.m = timm.create_model('efficientnet_b1',pretrained=True)
        self.fc = nn.Linear(self.m.classifier.out_features, num_of_classes)
    
    def forward(self, x):
        x = self.m(x)
        return self.fc(x)

In [8]:
class MaskStateDataSet(Dataset):
    def __init__(self,f_path,images_path,transform=None,train=True):
        self.f_path = f_path
        self.images_path = images_path
        self.transform = transform
        self.info = pd.read_csv('image_info.csv')
        self.y = self.info['mask_state']
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        y = torch.tensor(self.y[index])

        x = Image.open(self.f_path + self.images_path[index])
        if self.transform:
            x = self.transform(x)
        
        return x, y

In [9]:
state_clf = StateClassifier()
mask_state_data = MaskStateDataSet(f_path=folder_path,
                                        images_path=images_path,
                                        transform=transforms.Compose([Resize((260,260)),ToTensor(),
                                        Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2))]))

mask_state_data_loader = DataLoader(dataset=mask_state_data,batch_size=batch_size,
                                        shuffle=True,num_workers=4)

In [10]:
state_clf = state_clf.to(device)
criterion = nn.CrossEntropyLoss(weight = torch.tensor([1.,1.,6.]).to(device))
LEARNING_RATE = 0.001
EPOCHS = 10
optimizer = optim.Adam(state_clf.parameters(), lr=LEARNING_RATE)



for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in mask_state_data_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()        
        y_pred = state_clf(X_batch)
               
        loss = criterion(y_pred, y_batch)
        acc = accuracy(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(mask_state_data_loader):.5f} | Acc: {epoch_acc/len(mask_state_data_loader):.3f}')
    l = f'{epoch_loss/len(mask_state_data_loader):.5f}'
    a = f'{epoch_acc/len(mask_state_data_loader):.3f}'
    if not os.path.exists(MODEL_PATH):
        os.makedirs(MODEL_PATH)
    torch.save(state_clf.state_dict(), os.path.join(MODEL_PATH, f'state_model_{l}_{a}.pt'))

Epoch 001: | Loss: 0.05483 | Acc: 0.982
Epoch 002: | Loss: 0.02274 | Acc: 0.994
Epoch 003: | Loss: 0.00622 | Acc: 0.998
Epoch 004: | Loss: 0.00366 | Acc: 0.999
Epoch 005: | Loss: 0.00192 | Acc: 0.999


KeyboardInterrupt: 